# Graph Matching

You work at Facebook and Twitter, but there’s been a terrible incident on the Twitter end. All Twitter users’ names and handles have been somehow been deleted! Your bosses are furious and have tasked you with somehow recovering the lost information. How might you go about doing this? Luckily, you’ve been working hard and have somehow earned yourself this dual Facebook/Twitter gig, so you have a great resource at your disposal: the Facebook social network. You know all facebook users and who they are friends with, and since you’ve only lost the twitter usernames, you can still figure out which unnamed twitter users follow each other. You decide to use the Facebook network connectivity data to re-label the twitter social network. Alternatively, you can say the we are “aligning” Twitter based on Facebook. 

In the two social networks above, each user is a node and an edge exists if two users are friends. We'll define the facebook and twitter networks as $F$ and $T$ respectively, with associated adjacency matrices $A_F$ and $A_T$. Aligning the nodes of two networks is known as *Graph Matching*, because we are matching the node indices of one network (or, *graph*) to another. This can also be thought of as a mapping; that is, based on the neighbors of a node in $F$, you find a node in $T$ with the most similar neighborhood structure, then give the two nodes the same index. In other words, one of our Twitter users will be assigned the user name of the Facebook user with the most connections in common. This is done for the whole network, with the end result being that overall the structure is best preserved.

There are a ton of ways to match the nodes of two networks. In fact, for network pairs with $n$ nodes, there are $n!$ possible mappings; for example, when $n=100$, there are more than $10^{157}$ possible matchings. So how would we go about figuring out which mapping is best without the computationally gargantuan task of checking each one? First, we need a metric that tells us how similar two networks are to each other. For graph matching, this similarity metric is defined as $f(A, B) = ||A - B||_F^2$ for unweighted adjacency matrices $A$ and $B$. This quantity is known as the squared *Frobenius norm* (hence the subscript $F$) of the difference between $A$ and $B$. In other words, $f(A, B)$ is the sum of the squared elementwise differences between $A$ and $B$, or the quantity:
\begin{align*}
    f(A, B) &= ||A - B||_F^2 = \sum_{i, j}(a_{ij} - b_{ij})^2
\end{align*}
To understand this functionally, consider the best possible case where where the two networks (here they are two triangles) are identical: $A=B$. 
<div class="math">
\[
A = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
B = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0\end{array}
\right)\end{array}
\\
A-B =
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0\end{array}
\right)\end{array}
\\
||A - B||_F^2 = 0
\]
    </div>
As we seen above, the difference will be a matrix of all zeros, and taking the squared Frobenius norm will then yield $f(A,B) = 0$. This is because all of the element-wise differences $a_{ij} - b_{ij}$ are just zero, and hence both their square (and sum) will also be zero. Below we remove one edge from $B$,

<div class="math">
\[
A = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
B = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 0\\
1 & 0 & 0\end{array}
\right)\end{array}
\\
A-B =
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 0 & 0\\
0 & 0 & 1\\
0 & 1 & 0\end{array}
\right)\end{array}
\\
||A - B||_F^2 = 2
\]
    </div>

then $f(A,B) = 2$. If we consider the worst possible case (every edge in $A$ does not exist in $B$), 
<div class="math">
\[
A = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
B = 
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 0 & 0\\
0 & 0 & 0\\
0 & 0 & 0\end{array}
\right)\end{array}
\\
A-B =
\begin{array}{cc} &
\begin{array}{ccc} 0 & 1 & 2 \end{array}
\\
\begin{array}{ccc}
0 \\
1 \\
2 \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1\\
1 & 0 & 1\\
1 & 1 & 0\end{array}
\right)\end{array}
\\
||A - B||_F^2 = 6
\]
    </div>
then $f(A,B) = n(n - 1) = 3\cdot 2 = 6$. In this sense, the metric effectively counts the total number of disagreements in the adjacency matrices between $A$ and $B$. Thus, we want to find the mapping where $f(A, B)$ is as small as possible.


## Graph Matching Small Networks

Say we have the network pairs below, $T$ and $F$. They have four nodes each, $\{1, 2, 3, 4\}$ for $T$, and $\{a, b, c, d\}$ for $F$. In this case, the nodes represent people within the social networks, and the edges represent whether two people are connected on the social networking site. In this case, we will assume we have a node correspondance:
1. Person $0$ on Twitter is person $a$ on Facebook,
2. Person $1$ on Twitter is person $b$ on Facebook,
3. Person $2$ on Twitter is person $c$ on Facebook,
4. Person $3$ on Twitter is person $d$ on Facebook.
The corresponding adjacency matrices of the two networks are equal to each other when the nodes are laid out for $A_T$ as $\{0, 1, 2, 3\}$, and when the nodes are laid out for $A_F$ as $\{a,b,c,d\}$. 

![gm_11](gm_1.png)
<div class="math">
\[
A_T = 
\begin{array}{cc} &
\begin{array}{cccc} 0 & 1 & 2 & 3 \end{array}
\\
\begin{array}{cccc}
0 \\
1 \\
2 \\
3 \end{array}
&
\left(
\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
A_F = 
\begin{array}{cc} &
\begin{array}{cccc} a & b & c & d \end{array}
\\
\begin{array}{ccc}
a \\
b \\
c \\
d \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array} \\
|A_T - A_F| = \left(
\begin{array}{ccc}
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\end{array}
\right)
\]
    </div>
And as we learned above, therefore $f(A_T, A_B) = 0$. As we can see, when we compare the networks with the nodes ordered correctly (Person $1$ Twitter is the first node, Person $a$ from Facebook is its first node, Person $2$ from Twitter is the first node of $A_T$, Person $b$ from Facebook is the second node of $A_F$, so on and so forth). What we've done here by ordering the nodes in such tuples is implicitly made the statement that, there is something similar about the nodes $j$ between Twitter and Facebook, and therefore, ordering the adjacency matrices while obeying this similarity is going to give us a *low* edge disagreement (here, just zero). 

We can see this easily, by considering what happens if we reorder the nodes *simultaneously* for each network. Let's instead order the nodes for Twitter and Facebook as $\{2, 0, 1, 3\}$ and $\{c, a, b, d\}$, respectively. We will call these new adjacency matrices with the new node ordering $A_T''$ and $A_F''$ respectively:

<div class="math">
\[
A_T = 
\begin{array}{cc} &
\begin{array}{cccc} 2 & 0 & 1 & 3 \end{array}
\\
\begin{array}{cccc}
2 \\
0 \\
1 \\
3 \end{array}
&
\left(
\begin{array}{cccc}
0 & 1 & 0 & 1 \\
1 & 0 & 1 & 0\\
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0
\end{array}
\right)\end{array}
\quad \quad
A_F = 
\begin{array}{cc} &
\begin{array}{cccc} c & a & b & d \end{array}
\\
\begin{array}{ccc}
c \\
a \\
b \\
d \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 0 & 1 \\
1 & 0 & 1 & 0\\
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0
\end{array}
\right)\end{array} \\
|A_T - A_F| = \left(
\begin{array}{ccc}
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\end{array}
\right)
\]
    </div>

The idea is that, even though the ordering of the nodes is different, we have maintained that *like goes with like*: the first node for Twitter is node $2$ and the first node for Facebook is node $c$, the second node for Twitter is $0$ and the second node for Facebook is node $a$, so on and so forth. The node orderings preserve the *correspondance* between the nodes of Twitter with the nodes of Facebook.

Unfortunately, nothing in life nor network machine learning is ever this simple. The spatial layout of a network's nodes is arbirary, and in reality it can often be much harder to tell whether two networks are the same. For graph matching, we don't actually *know* the node correspondance between the two networks: we have no idea that person $0$ on Twitter is person $a$ on Facebook, person $1$ on Twitter is person $b$ on Facebook, so on and so forth.

For instance, we can swap the spatial location of nodes $c$ and $d$ in network $F$, as shown below. Even with such a small network, it's hard to tell whether the networks are the same. 

![gm_22](gm_2.png)

Further, let's say we just had an arbitrary ordering of the nodes for Facebook, which is now a little bit different from the one which we just saw. Let's imagine we have the nodes ordered instead as $\{a, b, d, c\}$ instead of $\{a, b, c, d\}$, like we did before, but Twitter's nodes are still ordered as $\{0, 1, 2, 3\}$. Now, the adjacency matrices won't be equal either, where we use $A_F'$ to denote the adjacency matrix for the Facebook network with this new node ordering:

<div class="math">
\[
A_T = 
\begin{array}{cc} &
\begin{array}{cccc} 0 & 1 & 2 & 3 \end{array}
\\
\begin{array}{cccc}
0 \\
1 \\
2 \\
3 \end{array}
&
\left(
\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
A_F' = 
\begin{array}{cc} &
\begin{array}{cccc} a & b & d & c \end{array}
\\
\begin{array}{ccc}
a \\
b \\
d \\
c \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0\\
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0\end{array}
\right)\end{array} \\
|A_T - A_F'| = \left(
\begin{array}{ccc}
0 & 0 & 1 & 1\\
0 & 0 & 1 & 1\\
1 & 1 & 0 & 0\\
1 & 1 & 0 & 0\end{array}
\right)
\]
    </div>
and we get that $f(A_T, A_F') = 8$, since there are $8$ entries which are different in the adjacency matrices between $A_T$ and $A_F$. This might seem a bit high, but note that because the network is undirected, adjecency disagreements are effectively counted twice (a single edge disagreement for an edge $(i,j)$ also yield a difference for edge $(j, i)$, since the adjacency matrix is symmetric). By comparing the nodes of Twitter and Facebook with the nodes *misaligned*, we have effectively *broken* the node correspondance between the nodes of Twitter and Facebook. 

In this sense, we can see how networks with a low number of edge disagreements might be considered to be better *matches* in terms of the node correspondance. When the nodes are *aligned* in a way which respects the node correspondance, identical networks have a low number of disagreements, and when the nodes are *aligned* in a way which disregards the node correspodance, the networks have a high number of disagreements. We will build upon this idea further, by exploring how to manipulate our adjacency matrices such that we can find alignments that match well. You can do this using something called a permutation matrix.

## Permutation Matrices

Permutation matrices are used to shuffle around the rows and columns of other matrices. A permutation matrix is a matrix of all ones and zeros, where each row and column adds up to one. In other words, each row has exactly one entry equal to one, with the rest being zeros; the same is true for the columns.


### Permutation matrices

Permutation matrices are commonly used as a method to move around the rows and columns of a square matrix. A **permutation matrix** is a matrix where, for every row and column, exactly one entry has a value of one. 

#### $PB$ moves the rows

Let's consider a matrix $B$ where all entries of the first row have a value of one, all entries of the second row have a value of two, all entries of the third row have a value of three, and all entries of the fourth row have a value of four. We can apply a permutation matrix $P$ to swap the rows around with the following heuristic. If the matrix $P$ has an entry $p_{ij}$ which is one, then in the resulting matrix, the row $i$ will be the row $j$ from the matrix we permuted. 

For instance, in the following example, the values $p_{12}$, $p_{23}$, $p_{34}$, and $p_{41}$ all have values of one, which means we will reorder the rows of $B$ so that $PB$ will have the top row being the second row from the original matrix (and will have a value of two), the second row will be the third row from the original matrix (and will have a value of three), the third row will be the fourth row from the original matrix (and will have a value of four), and the fourth row will be the first row from the original matrix (and will have a value of one. 

We apply this "row" permutation with the matrix multiplication $PB$:

In [ ]:
import numpy as np

B = np.array([[1,1,1,1],
              [2,2,2,2],
              [3,3,3,3],
              [4,4,4,4]])

P = np.array([[0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [1,0,0,0]])
# P * B
PB = P @ B

In [ ]:
from graspologic.plot import heatmap
import matplotlib.pyplot as plt
from graphbook_code import cmaps
fig, axs = plt.subplots(1, 3, figsize=(15, 12))
mtxs = [B, P, PB]
titles = ["Original matrix $B$", "Permutation Matrix $P$", "Row Permutation $PB$"]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, cmap=cmaps["sequential"], title = titles[i])
    ax.set_xlabel("Column")
    ax.set_ylabel("Row")
    ax.set_xticks([0.5,1.5,2.5,3.5])
    ax.set_yticks([0.5,1.5,2.5,3.5])
    ax.set_xticklabels([1,2,3,4])
    ax.set_yticklabels([1,2,3,4])

### $CP^\top$ moves the columns

Likewise, a column permutation behaves very similarly. Let's now consider a matrix $C$, where the first column has a value of one, the second column has a value of two, the third column has a value of three, and the fourth column has a value of four. We use the same permutation matrix, where here, $p_{ij}$ indicates that column $i$ of the new matrix will be column $j$ from the matrix before the permutation was applied. We apply the column permutation matrix as $CP^\top$:

In [ ]:
C = np.array([[1,2,3,4],
              [1,2,3,4],
              [1,2,3,4],
              [1,2,3,4]])

P = np.array([[0,1,0,0],
              [0,0,1,0],
              [0,0,0,1],
              [1,0,0,0]])
# C * P.T
CPt = C @ P.T

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 12))
mtxs = [C, P, CPt]
titles = ["Original matrix $C$", "Permutation Matrix $P$", r"Column Permutation $CP^\top$"]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, cmap=cmaps["sequential"], title = titles[i])
    ax.set_xlabel("Column")
    ax.set_ylabel("Row")
    ax.set_xticks([0.5,1.5,2.5,3.5])
    ax.set_yticks([0.5,1.5,2.5,3.5])
    ax.set_xticklabels([1,2,3,4])
    ax.set_yticklabels([1,2,3,4])

### $PDP^\top$ moves the rows and columns concurrently

As an interesting property of permutation matrices, we can apply these operations sequentially to reorder both the rows *and* columns of a matrix. Consider, for instance, a permutation matrix where row/column $1$ of the original matrix becomes row/column $2$ of the new matrix, and likewise, row/column $2$ of the original matrix becomes row/column $1$ of the new matrix. We'll consider a matrix $D$ where the first row and first column both have entries of all ones, and the rest of the matrix has the value zero. The original matrix and the permutation matrix look like this:

In [ ]:
D = np.array([[1,1,1,1],
              [1,0,0,0],
              [1,0,0,0],
              [1,0,0,0]])

P = np.array([[0,1,0,0],
              [1,0,0,0],
              [0,0,1,0],
              [0,0,0,1]])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 12))
mtxs = [D, P]
titles = ["Original matrix $D$", "Permutation Matrix $P$"]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, cmap=cmaps["sequential"], title = titles[i])
    ax.set_xlabel("Column")
    ax.set_ylabel("Row")
    ax.set_xticks([0.5,1.5,2.5,3.5])
    ax.set_yticks([0.5,1.5,2.5,3.5])
    ax.set_xticklabels([1,2,3,4])
    ax.set_yticklabels([1,2,3,4])

So, looking at the permutation matrix, if we were to apply this as a row or column permutation, row/column 3 and 4 will stay the same ($p_{33} = 1$ and $p_{44} = 1$, so row/column $3$ and $4$ will become row/column $3$ and $4$ respectively in the new representations), and row/column $1$ and $2$ would swap (row/column $1$ would become row/column $2$, and vice versa). Let's see what happens when we apply these sequentially:

In [ ]:
PD = P @ D
PDPt = PD @ P.T

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 14))
mtxs = [D, PD, PDPt]
titles = ["Original matrix $D$", "Permutate the rows $PD$", r"Permute the rows then the columns $PDP^\top$"]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, cmap=cmaps["sequential"], title = titles[i])
    ax.set_xlabel("Column")
    ax.set_ylabel("Row")
    ax.set_xticks([0.5,1.5,2.5,3.5])
    ax.set_yticks([0.5,1.5,2.5,3.5])
    ax.set_xticklabels([1,2,3,4])
    ax.set_yticklabels([1,2,3,4])

So, we take the original matrix, and first begin by swapping rows $1$ and $2$ to give us $PD$. Next, using this row-permuted matrix, we then permute the columns by swapping columns $1$ and $2$ of $PD$, to give us $PD^\top$. This shows that by using a permutation matrix to row and then column swap, we can reorder the rows and columns of $D$ simultaneously.

### Using concurrent row and column permutations on adjacency matrices

For our networks, remember that the adjacency matrix is the matrix $A$ where the entry $a_{ij}$ represents whether or not there is an edge between nodes $i$ and $j$. The key aspect is that the indexing for the adjacency matrix, $ij$, is an indexing over a single set: the nodes. This means that if we want to *reorder* the adjacency matrix by moving around the nodes, we need to move both the rows *and* the columns concurrently, since the *node ordering* is what is being permuted. If we had a permutation of the nodes given by $P$, we would correspondingly reorder the adjacency matrix by permuting the rows *and*
columns of $A$ by using $PAP^\top$.

### Permutation Matrices to Match Network

Next, we again consider the previous simple network example from Twitter and Facebook networks. Remember that we had two networks, where there was a node correspondance in that person $0$ from Twitter was the same as person $a$ from Facebook, person $1$ from Twitter was the same as the person $b$ from Facebook, so on and so forth. 

We will suppose that the ordering of the nodes from Twitter are given to us in order, $\{0, 1, 2, 3\}$, so in the ideal case, if the nodes from Facebook respect the node correspondance and the nodes are ordered $\{a, b, c, d\}$, we have that:


![gm_11](gm_1.png)
<div class="math">
\[
A_T = 
\begin{array}{cc} &
\begin{array}{cccc} 0 & 1 & 2 & 3 \end{array}
\\
\begin{array}{cccc}
0 \\
1 \\
2 \\
3 \end{array}
&
\left(
\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
A_F = 
\begin{array}{cc} &
\begin{array}{cccc} a & b & c & d \end{array}
\\
\begin{array}{ccc}
a \\
b \\
c \\
d \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array} \\
|A_T - A_F| = \left(
\begin{array}{ccc}
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\\
0 & 0 & 0 & 0\end{array}
\right)
\]
    </div>
and $f(A_T, A_F) = 0$.

However, if you remember, the problem was that we were *instead* given the nodes from Facebook in a different order; we were given the node ordering $\{a, b, d, c\}$ with adjacency matrix $A_F'$, leading to:

<div class="math">
\[
A_T = 
\begin{array}{cc} &
\begin{array}{cccc} 0 & 1 & 2 & 3 \end{array}
\\
\begin{array}{cccc}
0 \\
1 \\
2 \\
3 \end{array}
&
\left(
\begin{array}{cccc}
0 & 1 & 1 & 0\\
1 & 0 & 0 & 1\\
1 & 0 & 0 & 1\\
0 & 1 & 1 & 0\end{array}
\right)\end{array}
\quad \quad
A_F' = 
\begin{array}{cc} &
\begin{array}{cccc} a & b & d & c \end{array}
\\
\begin{array}{ccc}
a \\
b \\
d \\
c \end{array}
&
\left(
\begin{array}{ccc}
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0\\
0 & 1 & 0 & 1\\
1 & 0 & 1 & 0\end{array}
\right)\end{array} \\
|A_T - A_F'| = \left(
\begin{array}{ccc}
0 & 0 & 1 & 1\\
0 & 0 & 1 & 1\\
1 & 1 & 0 & 0\\
1 & 1 & 0 & 0\end{array}
\right)
\]
    </div>
    
and $f(A_T, A_F') = 8$. What we want to do is construct a permutation matrix $P$, which will keep nodes $a$ and $b$ in the same order, but swap nodes $c$ and $d$ in the node ordering. We can do this using exactly the strategy we developed above:

In [ ]:
AT = np.array([[0,1,1,0],
              [1,0,0,1],
              [1,0,0,1],
              [0,1,1,0]])
AFp = np.array([[0,1,0,1],
              [1,0,1,0],
              [0,1,0,1],
              [1,0,1,0]])

P = np.array([[1,0,0,0],
              [0,1,0,0],
              [0,0,0,1],
              [0,0,1,0]])

PAFpPt = P @ AFp @ P.T

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 15))

mtxs = [AT, AFp, PAFpPt]
titles = ["$A_T$", r"$A_F'$", r"$A_F = PA_F' P^\top$"]

node_odx = [[0,1,2,3], ["a", "b", "d", "c"], ["a", "b", "c", "d"]]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, title=titles[i], cmap=cmaps["sequential"])
    ax.set_xlabel("Node")
    ax.set_ylabel("Node")
    ax.set_xticks([0.5,1.5,2.5,3.5])
    ax.set_yticks([0.5,1.5,2.5,3.5])
    ax.set_xticklabels(node_odx[i])
    ax.set_yticklabels(node_odx[i])

As shown in the code block above, using a properly chosen permutation matrix $P$, we are able to recover the node correspondance between the networks for Twitter and Facebook. We obtain that for this particular choice of $P$, that $f(A_T, PA_F'P^\top) = 0$.

In this way, we will use this intuition to formulate the graph matching problem. For any two adjacency matrices $A, B$, we seek to minimize the cost function $g(A,B,P) = || A - PBP^\intercal||_F^2$ with the restriction that $P$ is a permuation matrix. You will notice that this is the definition of the Frobenius norm of the difference between $A$ and $B$, where you have now expanded $f$ to the function $g$ which includes a permutation of the rows and columns of $B$. This means that you want to figure out a way in which you can shuffle the rows and columns of $B$, such that it is as close as possible to $A$. In mathematics, the process of minimizing (or maximizing) a function based on a set of restrictions (called *constraints*) is known as *optimization*.

## Finding an Good Permutation with Gradient Descent Optimization

The algorithm used for solving graph matching optimization problem we described above is a variation of gradient descent.  The specifics of the algorithm are beyond the scope of this book, but for now you can simply imagine it as gradient descent. A gradient can be thought of as a vector valued slope; it is simply the slope of a function in all of it's dimensions, at a single point in space. Gradient Descent is a very common optimization method using to find optimal solutions for a wide range of problems. 

A simple way to think of the method is gravity.  Consider an inspector using a golf ball to find the lowest point when installing a drain. The ball rolls down hill until it comes to a stop; once it stops, we know we've found the lowest point. Gradient descent works in a similar way, taking steps in the direction of the local gradient with respect to some parameter. Once the gradient is zero, a local minimum has been found and the algorithm is stopped.

The main steps of a gradient descent method are choosing a suitable initial position (can be chosen randomly), then gradually improving the cost function one step at a time, until the function is changing by a very small amount, converging to a minimum. The main issue with gradient descent is that it does not guarantee that you will find a global minimum, only that you will find the local minimum of your initial position.

![grad_desc](grad_desc.png)

The image above is a simplification in two dimensions; the network functions we optimize over are n dimensional when matching networks with n nodes, making the problem incredibly difficult to solve. For this reason (among others outside of the scope of this book), the state-of-the-art graph matching algorithm is an approximation algorithm.


## Graph Matching with graspologic

For the example below, we will match two networks with a known node mapping that preserves a common network structure. To do this, we simulate a single Erdos-Reyni network, $A$, with six nodes and an edge probability of 0.5. Then, we generate $B$ by randomly permuting the node labels of $A$.

In [ ]:
from graspologic.simulations import er_np

n = 6
p = 0.5

# np.random.seed(1)
A = er_np(n=n, p=p)
node_shuffle_input = np.random.permutation(n)
B = A[node_shuffle_input][:, node_shuffle_input]
disagreements = np.linalg.norm(A - B)**2

In [ ]:
print("Number of adjecnecy disagreements: {:d}".format(int(disagreements)))

titles=['$A$, a realization of $ER_6(0.5)$', '$B$ ($A$ with the nodes randomly shuffled)']
fig, axs = plt.subplots(1, 2, figsize=(15, 15))
nodeAnames = [i for i in range(1, n+1)]
nodenames = [nodeAnames, [j+1 for j in node_shuffle_input]]
mtxs = [A, B]
for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, xticklabels=nodenames[i], cmap=cmaps["sequential"],
            yticklabels=nodenames[i], title=titles[i])
    ax.set_xlabel("Node")
    ax.set_ylabel("Node")

Below, we create a model to solve the Graph Matching Problem using the `GraphMatch` class. The model is then fit for the two networks A and B.

In [ ]:
from graspologic.match import GraphMatch

gmp = GraphMatch()
gmp = gmp.fit(A,B)
PBPt = B[gmp.perm_inds_][:, gmp.perm_inds_]
disagreements_after = np.linalg.norm(A - PBPt)**2
new_nodeodx = node_shuffle_input[gmp.perm_inds_]

In [ ]:
print("Number of adjacency disagreements: {:d}".format(int(disagreements_after)))


titles = ["$A$", "$B$ ($A$ shuffled)", "$PBP^\\top$, $B$ with nodes unsuffled", "$|A - PBP^\\top|$"]
fig, axs = plt.subplots(1, 4, figsize=(20, 20))
nodenames = [nodeAnames, [j+1 for j in node_shuffle_input], [j + 1 for j in new_nodeodx], ["" for i in nodeAnames]]
mtxs = [A, B, PBPt, np.abs(A - PBPt)]

for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, xticklabels=nodenames[i],
            cmap=cmaps["sequential"], yticklabels=nodenames[i], 
            title=titles[i])
    ax.set_xlabel("Node")
    ax.set_ylabel("Node")

The graph matching algorithm is able to successfully unshuffle $B$, with zero adjacency disagreements between $A$ and the matched $B$.

### Seeds

As mentioned previously, as networks become larger, they quickly become more difficult to match. One method to mitigate this difficulty is to use $\textit{seeds}$. Seeds are a subset of matches that we already know before we perform the graph matching. For example, if we are given two networks $T$ and $F$ with 300 nodes each, we might already know ten node matches between $T$ and $F$. Having this prior information greatly improves our ability to match the networks. 

## Seeded Graph Matching on Correlated Network Pairs

To demonstrate the effectiveness of Seeded Graph Matching (`SGM`), the algorithm will be applied on a pair of correlated SBM networks (undirected, no self loops), which is a simpler adaptation of the $\rho$-correlated $RDPG$ which we learned about in [Chapter 5](#link?). Like the $\rho$-correlated $RDPG$, the idea here is that we have two normal SBMs, but for any edge in the two networks $\mathbf a_{ij}^{(1)}$ and $\mathbf a_{ij}^{(2)}$, they will be correlated with correlation $\rho$. In words, if $\rho$ is positive, then if we know that $\mathbf a_{ij}^{(1)}$ has a value of one, then we have information to suggest that $\mathbf a_{ij}^{(2)}$ might be one too. In this case, we will have that $\mathbf A_1$ and $\mathbf A_2$ are $\rho-SBM_n(\vec z, B)$ where the networks are highly correlated, and $\rho = 0.9$. The block matrix is:
\begin{align*}
B &= \begin{bmatrix} 
0.7 & 0.3 & 0.4\\
0.3 & 0.7 & 0.3\\
0.4 & 0.3 & 0.7
\end{bmatrix}
\end{align*}
The first $100$ nodes in the network will be from community one, the second $100$ nodes in the network will be from community two, and the third $100$ nodes in the network will be from community three:

In [ ]:
import seaborn as sns
from graspologic.simulations import er_corr, sbm, sbm_corr
directed = False
loops = False
n_per_block = 75
n_blocks = 3
block_members = np.array(n_blocks * [n_per_block])
n_verts = block_members.sum()
rho = 0.9
block_probs = np.array([[0.7, 0.3, 0.4], [0.3, 0.7, 0.3], [0.4, 0.3, 0.7]])

A1, A2 = sbm_corr(block_members, block_probs, rho, directed=directed, loops=loops)
disagreements = np.linalg.norm(A1 - A2)**2

In [ ]:
print("Number of adjacency disagreements: {:d}".format(int(disagreements)))
fig, axs = plt.subplots(1, 3, figsize=(20, 10))
mtxs = [A1, A2, np.abs(A1 - A2)]
titles=["Network $A^{(1)}$", "Network $A^{(2)}$", "$|A^{(1)} - A^{(2)}|$"]
for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, title=titles[i], cmap=cmaps["sequential"])

To emphasize the effectiveness of `SGM`, as well as why having seeds is important, we will randomly shuffle the vertices of network $A^{(2)}$. We will call this version of $A^{(2)}$ after shuffling $A^{(2),s}$. This random permutation is stored, and unshuffled, such that we have available the optimal permutation that returns the original $A^{(2)}$. 

Here we see that after shuffling network $A^{(2)}$, there are many more edge disagreements, as expected.

In [ ]:
node_shuffle_input = np.random.permutation(n_verts)
A2_shuffled = A2[np.ix_(node_shuffle_input, node_shuffle_input)]
node_unshuffle_input = np.array(range(n_verts))
node_unshuffle_input[node_shuffle_input] = np.array(range(n_verts))
disagreements_shuffled = np.linalg.norm(A1 - A2_shuffled)**2

In [ ]:
print("Number of adjacency disagreements: {:d}".format(int(disagreements_shuffled)))

fig, axs = plt.subplots(1, 3, figsize=(20, 10))
mtxs = [A1, A2_shuffled, np.abs(A1 - A2_shuffled)]
titles=["Network $A^{(1)}$", "$A^{(2),s}$, Network $A^{(2)}$ shuffled", "$|A^{(1)} - A^{(2),s}|$"]
for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, title=titles[i], cmap=cmaps["sequential"])

### Unshuffling network 2 without seeds
First, we will run SGM on network $A^{(1)}$ and the shuffled network $A^{(2),s}$ with no seeds, and return the match ratio; that is, the fraction of vertices that have been correctly matched:

In [ ]:
# initialize sgm
sgm = GraphMatch()
# fit with A1 and shuffled A2
sgm = sgm.fit(A1, A2_shuffled)
# obtain unshuffled version of the shuffled A2
A2_unshuffled_noseed = A2_shuffled[np.ix_(sgm.perm_inds_, sgm.perm_inds_)]
# compute the match ratio
match_ratio_noseed = 1-(np.count_nonzero(abs(sgm.perm_inds_-node_unshuffle_input))/n_verts)
disagreements_noseed = np.linalg.norm(A1 - A2_unshuffled_noseed)**2

In [ ]:
print("Match Ratio with no seeds: {:.3f}".format(match_ratio_noseed))
print("Disagreements no seeds: {:d}".format(int(disagreements_noseed)))

fig, axs = plt.subplots(1, 3, figsize=(20, 10))
mtxs = [A1, A2_unshuffled_noseed, np.abs(A1 - A2_unshuffled_noseed)]
titles=["Network $A^{(1)}$", "$PA^{(2),s}P^\\top$, Network $A^{(2),s}$ unshuffled", "$|A^{(1)} - PA^{(2),s}P^\\top|$"]
for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, title=titles[i], cmap=cmaps["sequential"])

While the predicted permutation for $A^{(2),s}$ after unshuffling was successful in recovering the basic structure of the stochastic block model (i.e. network $A^{(1)}$ and network $PA^{(2),s}P^\top$ look qualitatively similar), we see that the number of edge disagreements between them is still quite high, and the match ratio of successfully unshuffled nodes is quite low. 

### Unshuffling network 2 with 10 seeds
Next, we will run SGM with 10 seeds randomly selected from the optimal permutation vector found ealier. Although 10 seeds is only about 4% of the 300 node network, we will observe below how much more accurate the matching will be compared to having no seeds:

In [ ]:
nseeds = 10  # the number of seeds to use
# select ten nodes at random from A1 which will serve as seeds
ref_seeds = np.sort(np.random.permutation(len(A1)-1)[:nseeds])
# identify the corresponding positions of these `nseeds` nodes
# in the shuffled version of A2
shuffled_seed_positions = np.array(node_unshuffle_input[ref_seeds])

# generate sgm instance
sgm = GraphMatch()
# run SGM with A2 and A2s, but provide the seed nodes from A1 as ref_seeds
# and the corresponding position of these seed nodes after shuffling as shuffled_seed_positions
sgm = sgm.fit(A1, A2_shuffled, ref_seeds, shuffled_seed_positions)
# unshuffle A2s using the optimal permutation of the nodes from sgm
A2_unshuffle_seeds = A2_shuffled[np.ix_(sgm.perm_inds_, sgm.perm_inds_)]

match_ratio_seeds = 1-(np.count_nonzero(abs(sgm.perm_inds_-node_unshuffle_input))/n_verts)
disagreements_seeds = np.linalg.norm(A1 - A2_unshuffle_seeds)**2

In [ ]:
print("Match Ratio with no seeds: {:.3f}".format(match_ratio_seeds))
print("Disagreements no seeds: {:d}".format(int(disagreements_seeds)))

fig, axs = plt.subplots(1, 3, figsize=(20, 10))
mtxs = [A1, A2_unshuffle_seeds, np.abs(A1 - A2_unshuffle_seeds)]
titles=["Network $A^{(1)}$", "$PA^{(2),s}P^\\top$, Network $A^{(2),s}$ unshuffled", "$|A^{(1)} - PA^{(2),s}P^\\top|$"]
for i, ax in enumerate(axs.flat):
    heatmap(mtxs[i], ax=ax, cbar=False, title=titles[i], cmap=cmaps["sequential"])

From the results above, we see that when running SGM on the same two networks, with no seeds there is match ratio is quite low. However including 10 seeds increases the match ratio to 100% (meaning that the shuffled $A^{(2),s}$ was completely correctly unshuffled).